# Timing Synchronization & Timing Error Detectors (TED)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from rfproto import filter, modulation, plot, sig_gen

## Timing Error Detector (TED)

* [Gardner Timing Error Detector: A Non-Data-Aided Version of Zero-Crossing Timing Error Detectors](https://wirelesspi.com/gardner-timing-error-detector-a-non-data-aided-version-of-zero-crossing-timing-error-detectors/)
* [Can we use Gardner timing error detector for multi level QAM or OFDM systems? - DSP Stack Exchange](https://dsp.stackexchange.com/questions/75927/can-we-use-gardner-timing-error-detector-for-multi-level-qam-or-ofdm-systems)
* [Symbol Synchronizer - Matlab Communications Toolbox](https://www.mathworks.com/help/comm/ref/comm.symbolsynchronizer-system-object.html)

## Polyphase Matched Filter for Timing Synchronization

Also see [the Multirate DSP page on Polyphase Filters](./Multirate_DSP.html#polyphase-filtering).

* [Symbol Synchronizer - Liquid SDR](https://liquidsdr.org/doc/symsync/)
* [Polyphase Clock Sync - GNU Radio](https://wiki.gnuradio.org/index.php/Polyphase_Clock_Sync)
* [Symbol Synchronization for SDR Using a Polyphase Filterbank Based on an FPGA](https://www.radioeng.cz/fulltexts/2015/15_03_0772_0782.pdf)
* [Simulating the TED gain for a polyphase matched filter](https://destevez.net/2020/02/simulating-the-ted-gain-for-a-polyphase-matched-filter/)

In [ ]:
# simulate random binary input values
num_symbols  = 2400
sym_rate     = 1e6 # Baseband symbol rate
# Generate random QPSK symbols
rand_symbols = np.random.randint(0, 4, num_symbols)

L  = 4               # Upsample ratio (Samples per Symbol)
fs = L * sym_rate    # Output sample rate (Hz)

rolloff          = 0.25 # Alpha of RRC
num_filt_symbols = 6    # Symbol length of RRC matched filter

qpsk_tx_filtered = sig_gen.gen_mod_signal(
    "QPSK",
    rand_symbols,
    fs,
    sym_rate,
    "RRC",
    rolloff,
    num_filt_symbols,
)

plot.IQ(qpsk_tx_filtered, alpha=0.1)
plt.show()

In [ ]:
plot.spec_an(qpsk_tx_filtered, fs=fs, fft_shift=True, show_SFDR=False, y_unit="dB", title="QPSK (4x SPS)")
plt.show()

In [ ]:
qpsk_tx_filtered = qpsk_tx_filtered[::2] # arbitrary downsample by to bring input to 2SPS (no need to filter in this sim)
plot.spec_an(qpsk_tx_filtered, fs=fs, fft_shift=True, show_SFDR=False, y_unit="dB", title="QPSK (2x SPS)")
plt.show()

In [ ]:
# Pass transmitted waveform through same RRC (matched filter)
L = 2
rrc_coef = filter.RootRaisedCosine(L, 1, rolloff, 2 * num_filt_symbols * L + 1)
rx_shaped = signal.lfilter(rrc_coef, 1, qpsk_tx_filtered)



In [ ]:
# adjust for best EVM, similar to slicer
timing_offset = 4
plot.IQ(rx_shaped[timing_offset::L], alpha=0.4)
plt.show()

In [ ]:
# The number of taps of this filter is based on how long you expect the channel to be; that is, 
# how many symbols do you want to combine to get the current symbols energy back, usually 5 to 10+
taps = 2 * num_filt_symbols * L + 1
# With 32 filters, you get a good enough resolution in the phase to produce very small, almost 
# unnoticeable, ISI. Going to 64 filters can reduce this more, but after that there is very little 
# gain for the extra complexity. Total prototype filter taps = taps * num_filters, since we're 
# instantiating segments of these taps into the filterbanks in such a way that each bank now 
# represents the filter at different phases, equally spaced at 2pi/N, where N is the number of filters.
num_filters = 31
polyphase_rrc_coef = filter.RootRaisedCosine(num_filters * L, 1, rolloff, taps * num_filters)

plot.filter_response(polyphase_rrc_coef)
plot.plt.show()

In [ ]:
h_poly_rrc = polyphase_rrc_coef.reshape(len(polyphase_rrc_coef)//num_filters, num_filters).T
print(np.shape(h_poly_rrc))

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
line, = ax.plot([], [], ".", alpha=0.3)
plt.axvline(x=0, color="orange")
plt.axhline(y=0, color="orange")
plt.margins(x=0)
plt.grid(True, linestyle="--")
plt.minorticks_on()
plt.tick_params(labelsize=8)
plt.xlabel("In-Phase (I)", fontsize=12)
plt.ylabel("Quadrature (Q)", fontsize=12)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xlim([-0.02, 0.02])
ax.set_ylim([-0.02, 0.02])

def update_plot(frame):
    poly_match_out = signal.lfilter(h_poly_rrc[frame], 1, qpsk_tx_filtered)
    line.set_xdata(np.real(poly_match_out[::2]))
    line.set_ydata(np.imag(poly_match_out[::2]))
    ax.set_title(f"Polyphase RRC Leg: {frame}")
    return line,

anim = FuncAnimation(fig, update_plot, frames=num_filters, interval=100, blit=True, repeat=True)
anim.save('iq_polyphase_timing.gif', writer='pillow')
plt.close()

<img src="iq_polyphase_timing.gif" width="750" align="center">

## References

* [Carrier and Timing Synchronization in Digital Modems - fred harris](https://s3.amazonaws.com/embeddedrelated/user/124841/synchronization_qualcomm_2018_4_11449.pdf)
* [On the Frequency Carrier Offset and Symbol Timing Estimation for CCSDS 131.2-B-1 High Data-Rate Telemetry Receivers](https://www.mdpi.com/1424-8220/21/9/2915)